In [ ]:
!pip install openai pinecone-client flask python-dotenv


In [ ]:
pip install flask pinecone-client google-cloud-aiplatform google-generativeai python-dotenv


In [ ]:
!pip install pinecone-client


In [ ]:
!pip uninstall pinecone-client


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pinecone/*
    /usr/local/lib/python3.11/dist-packages/pinecone_client-6.0.0.dist-info/*
Proceed (Y/n)? y
  Successfully uninstalled pinecone-client-6.0.0


In [ ]:
!pip install pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 9.0 MB/s eta 0:00:00


In [ ]:
import os
import json
from dotenv import load_dotenv
import pinecone
from google.cloud import aiplatform

# 1. Load env vars
# load_dotenv()
PINECONE_API_KEY = ""
PINECONE_ENVIRONMENT = ""
PINECONE_INDEX_NAME = ""
VERTEX_PROJECT_ID=""
VERTEX_LOCATION=""
EMBEDDING_MODEL=""


# 2. Init Pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)
index = pinecone.Index(PINECONE_INDEX_NAME)

# 3. Init Vertex AI
aiplatform.init(project=VERTEX_PROJECT_ID, location=VERTEX_LOCATION)
embedding_model = aiplatform.TextEmbeddingModel(model_name=EMBEDDING_MODEL)

def get_embedding(text):
    """Returns embedding vector from Vertex AI."""
    return embedding_model.get_embeddings([text])[0].values

# 4. Upsert flashcards
with open("data/flashcards.json") as f:
    flashcards = json.load(f)
to_upsert = []
for card in flashcards:
    text = f"{card['front']} {card['back']}"
    embedding = get_embedding(text)
    meta = {
        "type": "flashcard",
        "front": card["front"],
        "back": card["back"]
    }
    to_upsert.append((f"flashcard-{card['id']}", embedding, meta))
if to_upsert:
    index.upsert(to_upsert)
    print(f"Upserted {len(to_upsert)} flashcards.")

# 5. Upsert questions
with open("data/questions.json") as f:
    questions = json.load(f)
to_upsert = []
for q in questions:
    text = f"{q['question']} {' '.join(q.get('options', []))} {q.get('explanation', '')}"
    embedding = get_embedding(text)
    meta = {
        "type": "question",
        "question": q["question"],
        "options": q.get("options", []),
        "answer": q.get("answer", ""),
        "explanation": q.get("explanation", "")
    }
    to_upsert.append((f"question-{q['id']}", embedding, meta))
if to_upsert:
    index.upsert(to_upsert)
    print(f"Upserted {len(to_upsert)} questions.")

print("✅ All data upserted to Pinecone!")


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index',
            dimension=1536,
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
import json
import pinecone
from google.cloud import aiplatform

# # ==== CONFIGURATION ====
# PINECONE_API_KEY = "your_pinecone_api_key"
# PINECONE_ENVIRONMENT = "your_pinecone_environment"  # e.g. "us-east1-gcp"
# PINECONE_INDEX_NAME = "your_index_name"
# VERTEX_PROJECT_ID = "your-gcp-project-id"
# VERTEX_LOCATION = "us-central1"
# EMBEDDING_MODEL = "textembedding-gecko@003"

# ==== INIT ====
# Pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)
index = pinecone.Index(PINECONE_INDEX_NAME)

# Vertex AI
aiplatform.init(project=VERTEX_PROJECT_ID, location=VERTEX_LOCATION)
embedding_model = aiplatform.TextEmbeddingModel(model_name=EMBEDDING_MODEL)

def get_embedding(text):
    """Returns embedding vector from Vertex AI."""
    resp = embedding_model.get_embeddings([text])
    return resp[0].values

def upsert_items(json_path, obj_type):
    with open(json_path) as f:
        items = json.load(f)
    to_upsert = []
    for item in items:
        if obj_type == "flashcard":
            text = f"{item['front']} {item['back']}"
            meta = {
                "type": "flashcard",
                "front": item["front"],
                "back": item["back"]
            }
            item_id = f"flashcard-{item['id']}"
        else:  # question
            text = f"{item['question']} {' '.join(item.get('options', []))} {item.get('explanation', '')}"
            meta = {
                "type": "question",
                "question": item["question"],
                "options": item.get("options", []),
                "answer": item.get("answer", ""),
                "explanation": item.get("explanation", "")
            }
            item_id = f"question-{item['id']}"
        embedding = get_embedding(text)
        to_upsert.append((item_id, embedding, meta))
        print(f"Prepared {obj_type} {item_id}")

    if to_upsert:
        index.upsert(to_upsert)
        print(f"✅ Upserted {len(to_upsert)} {obj_type}s to Pinecone.")

# ==== MAIN ====
upsert_items("data/flashcards.json", "flashcard")
upsert_items("data/questions.json", "question")

print("🎉 All data upserted to Pinecone!")


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index',
            dimension=1536,
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
!pip install pinecone

In [ ]:
PINECONE_API_KEY = ""

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
index_name = "developer-quickstart-py"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [28]:
index = pc.Index(index_name)

In [23]:


# 2. Prepare records from flashcards
with open("/content/indian-medical-pg-flashcards.json") as f:
    data = json.load(f)

flashcards = data["flashcards"]   # <--- THIS is your list!

flashcard_records = [
    {
        "_id": f"flashcard-{card['id']}",
        "chunk_text": f"{card['front_content']} {card['back_content']}",
        "type": "flashcard",
        "front_content": card["front_content"],
        "back_content": card["back_content"],
        "subject": card.get("subject", ""),
        "topic": card.get("topic", ""),
        "card_type": card.get("card_type", ""),
        "card_variant": card.get("card_variant", ""),
        "image_urls": card.get("image_urls", None)
    }
    for card in flashcards
    if "front_content" in card and "back_content" in card
]




In [31]:
# 3. Prepare records from questions
with open("/content/indian-medical-pg-questions.json") as f:
    questions = json.load(f)
questions_list = questions["questions"]

question_records = [
    {
        "_id": f"question-{q['id']}",
        "chunk_text": f"{q['question']} {' '.join(q.get('options', []))} {q.get('explanation', '')}",
        "type": "question",
        "question": q.get("question", ""),
        "options": q.get("options", []),
        "answer": q.get("answer", ""),
        "explanation": q.get("explanation", "")
    }
    for q in questions_list
    if "question" in q
]

In [39]:
flashcard_records = [
    {
        "_id": f"flashcard-{card['id']}",
        "chunk_text": f"{card['front_content']} {card['back_content']}",
        "type": "flashcard",
        "front_content": card["front_content"],
        "back_content": card["back_content"],
        "subject": card.get("subject", ""),
        "topic": card.get("topic", ""),
        "card_type": card.get("card_type", ""),
        "card_variant": card.get("card_variant", ""),
        # Only include image_urls if not None
        # **({"image_urls": card["image_urls"]} if card.get("image_urls") is not None else {})
    }
    for card in flashcards
    if "front_content" in card and "back_content" in card
]


In [40]:
all_records = flashcard_records + question_records

In [41]:
BATCH_SIZE = 96  # Pinecone's serverless index limit

for i in range(0, len(all_records), BATCH_SIZE):
    batch = all_records[i:i+BATCH_SIZE]
    index.upsert_records(NAMESPACE, batch)
    print(f"Upserted records {i} to {i + len(batch)}")


Upserted records 0 to 96
Upserted records 96 to 192
Upserted records 192 to 288
Upserted records 288 to 384
Upserted records 384 to 480
Upserted records 480 to 576
Upserted records 576 to 672
Upserted records 672 to 768
Upserted records 768 to 864
Upserted records 864 to 960
Upserted records 960 to 1056
Upserted records 1056 to 1152
Upserted records 1152 to 1248
Upserted records 1248 to 1344
Upserted records 1344 to 1440
Upserted records 1440 to 1536
Upserted records 1536 to 1632
Upserted records 1632 to 1728
Upserted records 1728 to 1824
Upserted records 1824 to 1920
Upserted records 1920 to 2016
Upserted records 2016 to 2112
Upserted records 2112 to 2208
Upserted records 2208 to 2304
Upserted records 2304 to 2400
Upserted records 2400 to 2496
Upserted records 2496 to 2592
Upserted records 2592 to 2688
Upserted records 2688 to 2784
Upserted records 2784 to 2880
Upserted records 2880 to 2976
Upserted records 2976 to 3072
Upserted records 3072 to 3168
Upserted records 3168 to 3264
Upser

In [20]:
print(questions.keys())
# And maybe even print(questions) or print(list(questions.items())[:1])


dict_keys(['questions'])


In [44]:
query = "What is anatomy"

results = index.search(
    namespace=NAMESPACE,
    query={
        "top_k": 5,
        "inputs": {
            'text': query
        }
    }
)

print(results)

{'result': {'hits': [{'_id': 'flashcard-7f9cd053-f592-4a70-b8a6-f48bb47631a3',
                      '_score': 0.2758040428161621,
                      'fields': {'back_content': 'vulva; hymen',
                                 'card_type': 'premade',
                                 'card_variant': 'basic',
                                 'chunk_text': 'The _____ anatomically '
                                               'includes the skin and mucosa '
                                               'of the female genitalia '
                                               'external to the _____ (labia '
                                               'majora, labia minora, mons '
                                               'pubis, and vestibule) vulva; '
                                               'hymen',
                                 'front_content': 'The _____ anatomically '
                                                  'includes the skin and '
                     

In [15]:
print(flashcards.keys())


dict_keys(['metadata', 'flashcards'])


In [18]:
for i, card in enumerate(flashcards[:2]):
    print(f"Card {i}: {card}")


Card 0: {'id': '0014472d-7bd1-497e-ad62-af83e2d8ed8a', 'front_content': 'Griseofulvin will _____ serum levels of warfarin (increase or decrease)', 'back_content': 'decrease', 'subject': 'Pharmacology', 'topic': 'Antifungals', 'card_type': 'premade', 'card_variant': 'basic', 'image_urls': None}
Card 1: {'id': '001c4ae2-9786-4b39-b608-0508844c6fcc', 'front_content': 'Which NSAID at low doses inhibits uric acid excretion?_____', 'back_content': 'Aspirin', 'subject': 'Pharmacology', 'topic': 'Antimicrobials', 'card_type': 'premade', 'card_variant': 'basic', 'image_urls': None}


In [45]:
import re

def parse_user_query(user_query):
    """Extracts number and type (flashcard/question) from query."""
    query_lower = user_query.lower()
    n = 5  # Default value
    m = re.search(r'\b(\d+)\b', user_query)
    if m:
        n = int(m.group(1))
    if "flashcard" in query_lower:
        req_type = "flashcard"
    elif "question" in query_lower:
        req_type = "question"
    else:
        req_type = None
    return n, req_type

def show_search_results(user_query):
    n, req_type = parse_user_query(user_query)
    results = index.search(
        namespace=NAMESPACE,
        query={
            "top_k": n*2,   # Fetch more to filter
            "inputs": {"text": user_query}
        }
    )
    hits = results["result"]["hits"]
    filtered = []
    for hit in hits:
        typ = hit.get("fields", {}).get("type")
        if req_type is None or typ == req_type:
            filtered.append(hit)
        if len(filtered) == n:
            break

    # Print
    for i, hit in enumerate(filtered, 1):
        fields = hit["fields"]
        print(f"\nResult {i}:")
        print("-"*40)
        if fields.get("type") == "flashcard":
            print(f"Flashcard: {fields.get('front_content')}")
            print(f"Answer: {fields.get('back_content')}")
        elif fields.get("type") == "question":
            print(f"Question: {fields.get('question')}")
            print(f"Options: {fields.get('options')}")
            print(f"Answer: {fields.get('answer')}")
        print(f"Score: {hit.get('_score', '')}")

# Example:
show_search_results("Give me 3 flashcards about pharmacology")
show_search_results("5 questions on physiology")



Result 1:
----------------------------------------
Flashcard: First Aid Pharmacology: Autonomics ( blockers)  -blockers may have _____ adverse effects, including seizures, sedation, and sleep alterations
Answer: CNS
Score: 0.34405115246772766

Result 2:
----------------------------------------
Flashcard: First Aid Pharmacology: Autonomics ( blockers)  -blockers may have _____ adverse effects, including seizures, sedation, and sleep alterations
Answer: CNS
Score: 0.34405115246772766

Result 3:
----------------------------------------
Flashcard: First Aid Pharmacology: Neuro/Psych_____ is a local anesthetic that is associated with severe _____ toxicity
Answer: Bupivacaine; cardiovascular
Score: 0.3337332606315613


In [47]:
!pip install -U google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.24.0
    Uninstalling google-genai-1.24.0:
      Successfully uninstalled google-genai-1.24.0


In [ ]:
import pinecone
import google.generativeai as genai

# # === CONFIGURATION ===
# PINECONE_API_KEY = "YOUR_PINECONE_API_KEY"
# PINECONE_INDEX_NAME = "YOUR_INDEX_NAME"
# PINECONE_ENV = "YOUR_ENVIRONMENT"   # e.g. "us-east1-gcp"
# NAMESPACE = "medical-chatbot"
# GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"

# === INIT PINECONE ===
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(index_name)

# === INIT GEMINI ===
genai.configure(api_key=""
)
gemini_model = genai.GenerativeModel("gemini-1.5-flash-latest")

# === RETRIEVAL FUNCTION ===
def retrieve_pinecone(query, top_k=5):
    res = index.search(
        namespace=NAMESPACE,
        query={"top_k": top_k, "inputs": {"text": query}}
    )
    return res['result']['hits']

# === BUILD CONTEXT FUNCTION ===
def build_context(results, n=5):
    context = ""
    for i, hit in enumerate(results[:n], 1):
        fields = hit.get("fields", {})
        if fields.get("type") == "flashcard":
            context += f"\nFlashcard {i}: {fields.get('front_content')} (Answer: {fields.get('back_content')})"
        elif fields.get("type") == "question":
            context += f"\nQuestion {i}: {fields.get('question')} (Answer: {fields.get('answer')})"
        else:
            context += f"\n{i}: {fields.get('chunk_text', '')}"
    return context.strip()

# === RAG PIPELINE FUNCTION ===
def rag_respond(user_query, top_k=5):
    pinecone_results = retrieve_pinecone(user_query, top_k)
    context = build_context(pinecone_results, n=top_k)
    prompt = f"""You are a helpful medical study assistant.
Here are some relevant study materials for the user query: "{user_query}"
{context}

Now, answer the user's query or generate relevant flashcards/questions based on this information. Respond in a clear, concise way."""
    response = gemini_model.generate_content(prompt)
    return response.text

# === USAGE EXAMPLE ===
query = "Give me 3 flashcards about pharmacology"
print(rag_respond(query, top_k=3))


ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2630.90ms


Here are three flashcards about pharmacology, based on your provided text:

**Flashcard 1:**

* **Front:** What central nervous system (CNS) adverse effects can α-blockers cause?
* **Back:** Seizures, sedation, and sleep alterations.


**Flashcard 2:**

* **Front:** What type of blocker is associated with CNS adverse effects such as seizures, sedation, and sleep alterations?
* **Back:** α-blockers


**Flashcard 3:**

* **Front:** Bupivacaine is a local anesthetic associated with what type of severe toxicity?
* **Back:** Cardiovascular toxicity

